[View in Colaboratory](https://colab.research.google.com/github/RaulMedeiros/Deep_Transfer_Learning_Framework/blob/master/Deep_Transfer_Learning_Framework.ipynb)

**CONFIG DEEP FEATURE EXTRACTOR**

In [0]:
src_folder_id = '1xf3Qt48-btHzDw4vB4nlQOect0JQaytm'
out_folder_id = '1dQHvtUYX_HFXCH6lxSaYfN36_Vztncpv'

#Especifique a topologia a ser utilizada.
model_name_list = ['MobileNet',
                   'ResNet50',
                   'Xception',
                   'VGG16',
                   'VGG19',
                   'InceptionV3',
                   'InceptionResNetV2',
                   'NASNetMobile',
                   'NASNetLarge','DenseNet121','DenseNet169','DenseNet201']  

# #Especifique a topologia a ser utilizada.
model_name_list = ['DenseNet201']  

#Especifique o formato de saida
output_fmt = ['txt','npy']    

**Authenticate and create the PyDrive client**

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Donwload and Unzip Data.zip**

In [20]:
!rm -r data
!pip3 install -q pandas
import pandas as pd

def load_dataset(folder_id):
  """Download and unzip dataset"""   

  info = pd.DataFrame(drive.ListFile({'q': "'"+folder_id +"' in parents and trashed=false" }).GetList())  
  zip_id = info['id'].values[0]  
  downloaded = drive.CreateFile({'id': zip_id })
  
  downloaded.GetContentFile('./data.zip')  
  !unzip -q data.zip
  !rm data.zip
  !ls
  return True

#Load and Unconpress data
load_dataset(src_folder_id)

cuda
cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
cudnn-8.0-linux-x64-v7.1.tgz
data
datalab
libcudnn7_7.1.3.16-1+cuda8.0_amd64.deb
out


True

**Sample of files into class/folder 0**

In [21]:
!ls data/0

P01_01.JPG  P01_08.JPG	P01_15.JPG  P01_22.JPG	P01_29.JPG  P01_36.JPG
P01_02.JPG  P01_09.JPG	P01_16.JPG  P01_23.JPG	P01_30.JPG  P01_37.JPG
P01_03.JPG  P01_10.JPG	P01_17.JPG  P01_24.JPG	P01_31.JPG  P01_38.JPG
P01_04.JPG  P01_11.JPG	P01_18.JPG  P01_25.JPG	P01_32.JPG  P01_39.JPG
P01_05.JPG  P01_12.JPG	P01_19.JPG  P01_26.JPG	P01_33.JPG  P01_40.JPG
P01_06.JPG  P01_13.JPG	P01_20.JPG  P01_27.JPG	P01_34.JPG
P01_07.JPG  P01_14.JPG	P01_21.JPG  P01_28.JPG	P01_35.JPG


**Install Libraries **

In [0]:
# https://keras.io/
!pip install -q keras
import keras

# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

from IPython.display import clear_output

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import keras
from keras import backend as tf
from scipy import misc

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetMobile
from keras.applications.nasnet import NASNetLarge
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201

from keras.applications.inception_resnet_v2 import preprocess_input as ppi_inceptionresnet_v2
from keras.applications.vgg16 import preprocess_input as ppi_vgg16
from keras.applications.vgg19 import preprocess_input as ppi_vgg19
from keras.applications.xception import preprocess_input as ppi_xception
from keras.applications.resnet50 import preprocess_input as ppi_resnet50
from keras.applications.inception_v3 import preprocess_input as ppi_inception_v3
from keras.applications.mobilenet import preprocess_input as ppi_mobilenet
from keras.applications.nasnet import preprocess_input as ppi_nasnet
from keras.applications.densenet import preprocess_input as ppi_densenet


from keras.preprocessing import image
from keras.layers import Input

np.set_printoptions(threshold=np.nan)
keras.backend.backend()

import time
current_milli_time = lambda: int(round(time.time() * 1000))

In [0]:
def buildModel(modelName,pooltype='max'):
    if ('VGG16' is modelName):
        model = VGG16(weights='imagenet',pooling=pooltype, include_top=False)    
    if ('VGG19' is modelName):
        model = VGG19(weights='imagenet',pooling=pooltype, include_top=False)
    if ('MobileNet' is modelName):
        input_tensor = Input(shape=(224,224,3))
        model = MobileNet(weights='imagenet',pooling=pooltype,input_shape=(224,224,3),include_top=False)  
    if ('ResNet50' is modelName):
        model = ResNet50(weights='imagenet',pooling=pooltype, include_top=False)       
    if ('InceptionV3' is modelName):
        model = InceptionV3(weights='imagenet',pooling=pooltype, include_top=False)
    if ('Xception' is modelName):
        model = Xception(weights='imagenet',pooling=pooltype, include_top=False)
    if ('InceptionResNetV2' is modelName):
        model = InceptionResNetV2(weights='imagenet',pooling=pooltype, include_top=False)
    if ('NASNetMobile' is modelName):
        model = NASNetMobile(weights='imagenet',pooling=pooltype, include_top=False)
    if ('NASNetLarge' is modelName):
        model = NASNetLarge(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet121' is modelName):
        model = DenseNet121(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet169' is modelName):
        model = DenseNet169(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet201' is modelName):
        model = DenseNet201(weights='imagenet',pooling=pooltype, include_top=False)    
                
    if (modelName in ['InceptionV3','Xception','InceptionResNetV2']):
        targetSize = 299  
    elif(modelName in ['NASNetLarge']):
        targetSize = 331
    else:    
        targetSize = 224   
    
    return model,targetSize
  
def extractDeepFeatures(file_id,model,modelName,target_size):
    
    x = load_img(file_id,target_size)
    x = image.img_to_array(x)    
    x = np.expand_dims(x, axis=0)

    if (modelName is 'VGG16'):
        x = ppi_vgg16(x)        
    if (modelName is 'VGG19'):
        x = ppi_vgg19(x)
    if (modelName is 'ResNet50'):
        x = ppi_resnet50(x)
    if (modelName is 'MobileNet'):
        x = ppi_mobilenet(x)
    if (modelName is 'Xception'):
        x = ppi_xception(x) 
    if (modelName is 'InceptionV3'):
        x = ppi_inception_v3(x)
    if (modelName is 'InceptionResNetV2'):
        x = ppi_inceptionresnet_v2(x)
    if (modelName is 'NASNetMobile'):
        x = ppi_nasnet(x)
    if (modelName is 'NASNetLarge'):
        x = ppi_nasnet(x)
    if (modelName is 'DenseNet121'):
        x = ppi_densenet(x)
    if (modelName is 'DenseNet169'):
        x = ppi_densenet(x)
    if (modelName is 'DenseNet201'):
        x = ppi_densenet(x)

    timeStart = current_milli_time()
    features = model.predict(x).reshape(-1)
    process_time = current_milli_time()-timeStart
    
#     print('Load process: Complete',(current_milli_time()-timeStart)/1000)

    return features,process_time

def load_img(filepath,target_size):
    """Load dataset"""
    #LOAD image to be deep extracted         
    if (filepath.endswith('jpg') or filepath.endswith('JPG') ):
        img = cv2.imread((filepath))
    elif (filepath.endswith('txt')):
        img = np.loadtxt(filepath).copy()
    elif (filepath.endswith('npy')):
        img = np.load(filepath).copy()
    elif (filepath.endswith('png') or filepath.endswith('PNG')):
        img = misc.imread(filepath).copy()
    img = cv2.resize(img, (target_size,target_size), interpolation = cv2.INTER_NEAREST)
    return img

**Iterate through each model and each image to extract deep features **

In [0]:
def deep_extractor(model_name,numOfClasses,out_folder_id,output_fmt=['npy'],
              srcPath='./data',outPath='./out'):
  
  print(model_name)
  cont = 0

  # For each class    
  model,targetSize = buildModel(model_name)
  data = []
  process_time_list= []

  for fileIdx in range(numOfClasses):
    folderPath = srcPath+'/'+str(fileIdx)

    # For each sample
    for subdir, dirs, files in os.walk(folderPath):
      for idx,name in enumerate(files):
        filePath = subdir + os.sep + name
        cont=cont+1

        clear_output()             
        print(cont,filePath,model_name) 

        features,process_time = extractDeepFeatures(filePath,model,model_name,targetSize)
        process_time_list.append(process_time)
        
        features = np.hstack((features,fileIdx))
        data.append(features)
        
  # Creates a folder if don't exist
  os.makedirs(outPath+'/', exist_ok=True) 
  # Saves a txt file w/ deep features
  if('txt' in output_fmt):
    np.savetxt(outPath+'/'+model_name+'.txt',data,fmt="%.8f")
    #Upload txt file to folder (by id)
    file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                          "id": out_folder_id}]})
    file.SetContentFile(outPath+'/'+model_name+'.txt')
    file.Upload()

  #Saves a npy file w/ deep features
  if('npy' in output_fmt):
    np.save(outPath+'/'+model_name+'.npy',data)      
    #Upload npy file to folder (by id)
    file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                          "id": out_folder_id}]})
    file.SetContentFile(outPath+'/'+model_name+'.npy')
    file.Upload() 

  #Save process time into a file
  np.savetxt(outPath+'/'+model_name+'_time.txt',process_time_list,fmt="%d")
  #Upload txt file to folder (by id)
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                        "id": out_folder_id}]})
  file.SetContentFile(outPath+'/'+model_name+'_time.txt')
  file.Upload()
  return True


In [25]:
%%time

## Especifique o caminho da pasta onde estão as imagens a serem processadas.
srcPath = './data'

## Especifique o caminho da pasta onde serão armazenadas as caracteristicas extridas.
outPath = './out'                         

#Encontra o numero de classes do problema.
numOfClasses = len(glob.glob('./'+srcPath+'/*'))

# For each deep model    
for model_name in model_name_list:
  deep_extractor(model_name,numOfClasses,out_folder_id,output_fmt,srcPath,outPath)
  

600 ./data/14/P15_39.JPG DenseNet201
CPU times: user 5min 43s, sys: 39.4 s, total: 6min 23s
Wall time: 6min 17s


In [0]:
# %%time
# ## Especifique o caminho da pasta onde estão as imagens a serem processadas.
# srcPath = './data'

# ## Especifique o caminho da pasta onde serão armazenadas as caracteristicas extridas.
# outPath = './out'                         

# #Encontra o numero de classes do problema.
# numOfClasses = len(glob.glob('./'+srcPath+'/*'))

# # For each deep model    
# for model_name in model_name_list:
#     print(model_name)
#     cont = 0
    
#     # For each class    
#     model,targetSize = buildModel(model_name)
#     data = []
    
#     for fileIdx in range(numOfClasses):
#         folderPath = srcPath+'/'+str(fileIdx)
# #         print(folderPath)

#         # For each sample
#         for subdir, dirs, files in os.walk(folderPath):
#             for idx,name in enumerate(files):
#                 filePath = subdir + os.sep + name
#                 cont=cont+1
                
#                 clear_output()             
#                 print(cont,filePath,model_name)    
                
#                 features = extractDeepFeatures(filePath,model,model_name,targetSize)
#                 features = np.hstack((features,fileIdx))
#                 data.append(features)

#     # Creates a folder if don't exist
#     os.makedirs(outPath+'/', exist_ok=True) 
#     # Saves a txt file w/ deep features
#     if('txt' in output_fmt):
#       np.savetxt(outPath+'/'+model_name+'.txt',data,fmt="%.8f")
#       #Upload txt file to folder (by id)
#       file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": out_folder_id}]})
#       file.SetContentFile(outPath+'/'+model_name+'.txt')
#       file.Upload()
      
#     #Saves a npy file w/ deep features
#     if('npy' in output_fmt):
#       np.save(outPath+'/'+model_name+'.npy',data)      
#       #Upload npy file to folder (by id)
#       file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": out_folder_id}]})
#       file.SetContentFile(outPath+'/'+model_name+'.npy')
#       file.Upload() 
   
# #     Delete Files    
# #     filepath = outPath+'/'+model_name+'.txt
# #     !rm filepath
# #     filepath = outPath+'/'+model_name+'.npy'
# #     !rm filepath

In [0]:
print('Done')

Done
